In [19]:
from selenium import webdriver
#import chromedriver_autoinstaller
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd

def get_prod_items(pro_items):    #크롤링 할 컨텐츠
    prod_data = []
    
    for prod_item in prod_items:
        try:  #상품명 크롤링
            product_name = prod_item.select('p.prod_name > a')[0].text.strip()
            product_name = product_name.replace(", 자급제", "")
            product_name_list = product_name.split()
            del product_name_list[0]
            product_name = ' '.join(product_name_list)
        except:
            product_name = ""

        try:  #상품 크롤링
            product_list = prod_item.select('div.spec_list')[0].text.strip()
            product_list = product_list.replace('보안/기능', '보안 및 기능')
            product_detail_list = product_list.split('/')
            
            for i in range(len(product_detail_list)):
                if '출시OS' in product_detail_list[i]:
                    idx_os = i
                elif '화면정보' in product_detail_list[i]:
                    idx_scr = i
                elif '시스템' in product_detail_list[i]:
                    idx_sys = i
                elif '통신' in product_detail_list[i]:
                    idx_conn = i
                elif '카메라' in product_detail_list[i]:
                    idx_cam = i
                elif '사운드' in product_detail_list[i]:
                    idx_sound = i
                elif '보안 및 기능' in product_detail_list[i]:
                    idx_func = i
                elif '배터리' in product_detail_list[i]:
                    idx_bat = i
                elif '규격' in product_detail_list[i]:
                    idx_size = i
                elif '벤치마크' in product_detail_list[i]:
                    idx_bench = i
        
            release_os_list = product_detail_list[idx_os]
            release_os = ''.join(release_os_list)
            release_os = release_os.replace(' 출시OS: ','').replace(' ', '')
        
            screen_info_list = product_detail_list[idx_scr:idx_sys]
            screen_info = ''.join(screen_info_list)
            screen_info = screen_info.replace(' 화면정보 ','').replace('  ', '/ ')
        
            system_list = product_detail_list[idx_sys:idx_conn]
            ram_list = []
            mem_list = []
            
            for j in system_list:
                if '램' in j:
                    ram_list += j
                    system_list.remove(j)
            for k in system_list:
                if '내장' in k:
                    mem_list += k
                    system_list.remove(k)
                    
            system = ''.join(system_list)
            ram = ''.join(ram_list)
            mem = ''.join(mem_list)
            system = system.replace(' 시스템 ','').replace('  ', '/ ')
            ram = ram.replace('램:', '')
            mem = mem.replace('내장:', '')
            
            connect_list = product_detail_list[idx_conn:idx_cam]
            connect = ''.join(connect_list)
            connect = connect.replace(' 통신 ','').replace('  ', '/ ')
        
            camera_list = product_detail_list[idx_cam:idx_sound]
            camera = ''.join(camera_list)
            camera = camera.replace(' 카메라 ','').replace('  ', '/ ')
        
            sound_list = product_detail_list[idx_sound:idx_func]
            sound = ''.join(sound_list)
            sound = sound.replace(' 사운드 ','').replace('  ', '/ ')
        
            function_list = product_detail_list[idx_func:idx_bat]
            function = ''.join(function_list)
            function = function.replace(' 보안 및 기능 ','').replace('  ', '/ ')
        
            battery_list = product_detail_list[idx_bat:idx_size]
            battery = ''.join(battery_list)
            battery = battery.replace(' 배터리 ','').replace('  ', '/ ')
        
            size_list = product_detail_list[idx_size:idx_bench]
            size = ''.join(size_list)
            size = size.replace(' 규격 ','').replace('  ', '/ ')
        
            bench_mark_list = product_detail_list[idx_bench:]
            bench_mark = ''.join(bench_mark_list)
            bench_mark = bench_mark.replace(' 벤치마크 ','').replace('  ', '/ ')
        except:
            size_list = product_detail_list[idx_size:]
            size = ''.join(size_list)
            size = size.replace(' 규격 ','').replace('  ', '/ ')
            bench_mark = ""

        try:
            img_link = 'http:' + prod_item.select_one('div.thumb_image > a > img').get('data-original')
        except:
            img_link = 'http:' + prod_item.select_one('div.thumb_image > a > img').get('src')

        mylist = [product_name, release_os, screen_info, system, ram, mem, connect, camera, sound, function, battery, size, bench_mark, img_link]
        
        if mylist[0]:
            prod_data.append(mylist)
    
    return(prod_data)

# 다나와 사이트 검색
 
options = Options()
options.add_argument('headless'); # headless는 화면이나 페이지 이동을 표시하지 않고 동작하는 모드

#chromedriver_autoinsttraller.install()
driver = webdriver.Chrome(options=options)
driver.implicitly_wait(5)
driver.set_window_size(1920,1280)

url = 'https://prod.danawa.com/list/?cate=12215709'  #핸드폰 자급제
driver.get(url)
curPage = 1  #시작 페이지
totalPage = 6  #총 페이지

prod_detail_total = []

while curPage <= totalPage:
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    prod_items = soup.select('li.prod_item.prod_layer')
    print('----- Current Page : {}'.format(curPage), '------')

    prod_item_list = get_prod_items(prod_items)
    prod_detail_total.append(prod_item_list)

    curPage += 1

    if curPage > totalPage:
        print('Crawling succeed!')
        break

    cur_css = 'div.number_wrap > a:nth-child({})'.format(curPage)
    WebDriverWait(driver,3).until(EC.presence_of_element_located((By.CSS_SELECTOR,cur_css))).click()    #페이지 넘기기

    del soup

    time.sleep(3)

driver.close()

total = []
for temp in prod_detail_total:
    total += temp
prod_detail_total = total

data = pd.DataFrame(prod_detail_total)

data.columns = ['상품명', '출시OS', '화면정보', '시스템', '램', '내장메모리', '통신', '카메라', '사운드', '보안/기능', '배터리', '규격', '벤치마크', '이미지']

----- Current Page : 1 ------
----- Current Page : 2 ------
----- Current Page : 3 ------
----- Current Page : 4 ------
----- Current Page : 5 ------
----- Current Page : 6 ------
Crawling succeed!


In [21]:
data = data.replace(", 공기계", "", regex=True).drop_duplicates(['상품명'])

In [22]:
data.reset_index(inplace = True)

In [23]:
data = data.drop("index", axis = 1)

In [20]:
data

,상품명,출시OS,화면정보,시스템,램,내장메모리,통신,카메라,사운드,보안/기능,배터리,규격,벤치마크,이미지
0,갤럭시S23 256GB,안드로이드13,"15.49cm(6.1"")/ D-AMOLED/ 2340x1080/ 422ppi/ 12...",스냅드래곤8 Gen2/ 4nm/ microSD 미지원,8GB,256GB,5G+LTE/ Wi-Fi6e/ 블루투스v5.3/ 나노유심/ 듀얼유심지원,"후면:5,000만화소+1,200만화소+1,000만화소/ 전면:1,200만화소/ 동영...",스피커:스테레오(2개),지문인식(온스크린)/ 얼굴인식/ 빅스비/ 삼성페이/ IP68/ GPS/ NFC/ O...,"USB타입C/ 3,900mAh/ 충전지원: 최대25W/ Qi무선충전: 최대10W",가로:70.9mm/ 세로:146.3mm/ 두께:7.6mm/ 무게:167g,GeekBench V5: 싱글:1518/ 멀티:4743,http://img.danawa.com/prod_img/500000/687/791/...
1,갤럭시A34 128GB,안드로이드13,"16.76cm(6.6"")/ S-AMOLED/ 2340x1080/ 390ppi/ 12...",디멘시티1080/ 6nm/ microSD:최대1TB,6GB,128GB,5G+LTE/ Wi-Fi5/ 블루투스v5.3/ 나노유심,"후면:4,800만화소+800만화소+500만화소/ 전면:1,300만화소/ 동영상:4k...",스피커:스테레오(2개),지문인식(온스크린)/ 빅스비/ 삼성페이/ IP67/ GPS/ NFC/ OTG,"USB타입C/ 5,000mAh/ 충전지원: 최대25W",가로:78.1mm/ 세로:161.3mm/ 두께:8.2mm/ 무게:199g,,http://img.danawa.com/prod_img/500000/337/751/...
2,갤럭시S23 울트라 256GB,안드로이드13,"17.27cm(6.8"")/ D-AMOLED/ 3088x1440/ 501ppi/ 12...",스냅드래곤8 Gen2/ 4nm/ microSD 미지원,12GB,256GB,5G+LTE/ Wi-Fi6e/ 블루투스v5.3/ 나노유심/ 듀얼유심지원,"후면:2억 화소+1,000만화소+1,000만화소+1,200만화소/ 전면:1,200만...",스피커:스테레오(2개),지문인식(온스크린)/ 빅스비/ 삼성페이/ IP68/ GPS/ NFC/ OTG/ AO...,"USB타입C/ 5,000mAh/ 충전지원: 최대45W/ Qi무선충전: 최대15W",가로:78.1mm/ 세로:163.4mm/ 두께:8.9mm/ 무게:233g,GeekBench V5: 싱글:1504/ 멀티:4580,http://img.danawa.com/prod_img/500000/722/768/...
3,갤럭시S23 울트라 512GB,안드로이드13,"17.27cm(6.8"")/ D-AMOLED/ 3088x1440/ 501ppi/ 12...",스냅드래곤8 Gen2/ 4nm/ microSD 미지원,12GB,512GB,5G+LTE/ Wi-Fi6e/ 블루투스v5.3/ 나노유심/ 듀얼유심지원,"후면:2억 화소+1,000만화소+1,000만화소+1,200만화소/ 전면:1,200만...",스피커:스테레오(2개),지문인식(온스크린)/ 빅스비/ 삼성페이/ IP68/ GPS/ NFC/ OTG/ AO...,"USB타입C/ 5,000mAh/ 충전지원: 최대45W/ Qi무선충전: 최대15W",가로:78.1mm/ 세로:163.4mm/ 두께:8.9mm/ 무게:233g,GeekBench V5: 싱글:1504/ 멀티:4580,http://img.danawa.com/prod_img/500000/361/907/...
4,아이폰14 프로 256GB,iOS16,"15.49cm(6.1"")/ OLED/ 2556x1179/ 460ppi/ 120Hz/...",A16 Bionic/ 4nm/ microSD 미지원,6GB,256GB,5G+LTE/ Wi-Fi6/ 블루투스v5.3/ 나노+eSIM,"후면:4,800만화소+1,200만화소+1,200만화소/ 전면:1,200만화소+SL ...",스피커:스테레오(2개),FaceID/ Siri/ 애플페이/ IP68/ GPS/ NFC,"라이트닝8핀/ 3,200mAh/ 맥세이프: 15W, Qi무선충전: 최대7.5W",가로:71.5mm/ 세로:147.5mm/ 두께:7.9mm/ 무게:206g,,http://img.danawa.com/prod_img/500000/393/816/...
5,갤럭시S23 플러스 256GB,안드로이드13,"16.76cm(6.6"")/ D-AMOLED/ 2340x1080/ 393ppi/ 12...",스냅드래곤8 Gen2/ 4nm/ microSD 미지원,8GB,256GB,5G+LTE/ Wi-Fi6e/ 블루투스v5.3/ 나노유심/ 듀얼유심지원,"후면:5,000만화소+1,200만화소+1,000만화소/ 전면:1,200만화소/ 동영...",스피커:스테레오(2개),지문인식(온스크린)/ 얼굴인식/ 빅스비/ 삼성페이/ IP68/ GPS/ NFC/ O...,"USB타입C/ 4,700mAh/ 충전지원: 최대45W/ Qi무선충전: 최대10W",가로:76.2mm/ 세로:157.8mm/ 두께:7.6mm/ 무게:195g,GeekBench V5: 싱글:1485/ 멀티:4844,http://img.danawa.com/prod_img/500000/302/768/...
6,아이폰14 프로 맥스 256GB,iOS16,"17.01cm(6.7"")/ OLED/ 2796x1290/ 460ppi/ 120Hz/...",A16 Bionic/ 4nm/ microSD 미지원,6GB,256GB,5G+LTE/ Wi-Fi6/ 블루투스v5.3/ 나노+eSIM,"후면:4,800만화소+1,200만화소+1,200만화소/ 전면:1,200만화소+SL ...",스피커:스테레오(2개),FaceID/ Siri/ 애플페이/ IP68/ GPS/ NFC,"라이트닝8핀/ 4,323mAh/ 맥세이프: 15W, Qi무선충전: 최대7.5W",가로:77.6mm/ 세로:160.7mm/ 두께:7.9mm/ 무게:240g,,http://img.danawa.com/prod_img/500000/720/816/...
7,갤럭시Z 플립4 256GB,안드로이드12,"17.01cm(6.7"")/ D-AMOLED/ 2640x1080/ 425ppi/ 12...",스냅드래곤8+ Gen1/ 4nm/ microSD 미지원,8GB,256GB,5G+LTE/ Wi-Fi6/ 블루투스v5.2/ 나노+eSIM,"후면:1,200만화소+1,200만화소/ 전면:1,000만화소/ 동영상:4K@60fp...",스피커:스테레오(2개),지문인식(측면)/ 삼성페이/ IPX8/ GPS/ NFC/ OTG,"USB타입C/ 3,700mAh/ 충전지원: 최대25W/ Qi무선충전: 최대15W",가로:71.9mm/ 세로:165.2mm/ 두께:6.9mm/ 무게:187g,,http://img.danawa.com/prod_img/500000/816/621/...
8,갤럭시S22 256GB,안드로이드12,"15.49cm(6.1"")/ D-AMOLED/ 2340x1080/ 422ppi/ 12...",스냅드래곤8 Gen1/ 4nm,8GB,256GB,5G+LTE/ Wi-Fi6/ 블루투스v5.2/ 나노유심,"후면:5,000만화소+1,200만화소+1,000만화소/ 전면:1,000만화소/ 광학...",,지문인식(온스크린)/ 빅스비/ 삼성페이/ IP68/ GPS/ NFC/ OTG,"USB타입C/ 3,700mAh/ 충전지원: 최대25W/ Qi무선충전: 최대15W",가로:70.6mm/ 세로:146mm/ 두께:7.6mm/ 무게:167g,3D마크: 와일드 라이프:9005,http://img.danawa.com/prod_img/500000/739/291/...
9,아이폰14 프로 128GB,iOS16,"15.49cm(6.1"")/ OLED/ 2556x1179/ 460ppi/ 120Hz/...",A16 Bionic/ 4nm/ microSD 미지원,6GB,128GB,5G+LTE/ Wi-Fi6/ 블루투스v5.3/ 나노+eSIM,"후면:4,800만화소+1,200만화소+1,200만화소/ 전면:1,200만화소+SL ...",스피커:스테레오(2개),FaceID/ Siri/ 애플페이/

In [16]:
data.drop_duplicates(['시스템'])['시스템']

0                스냅드래곤8 Gen2/ 4nm/ microSD 미지원 
1                 디멘시티1080/ 6nm/ microSD:최대1TB 
4                 A16 Bionic/ 4nm/ microSD 미지원 
7               스냅드래곤8+ Gen1/ 4nm/ microSD 미지원 
8                             스냅드래곤8 Gen1/ 4nm 
11                엑시노스1280/ 5nm/ microSD:최대1TB 
12                A15 Bionic/ 5nm/ microSD 미지원 
13               스냅드래곤8 Gen1/ 4nm/ microSD 미지원 
16                스냅드래곤680/ 6nm/ microSD:최대1TB 
18                엑시노스1380/ 5nm/ microSD:최대1TB 
25                 엑시노스850/ 8nm/ microSD:최대1TB 
31                  스냅드래곤888/ 5nm/ microSD 미지원 
34                     스냅드래곤732G/ 8nm/ microSD 
37                스냅드래곤778G+/ 6nm/ microSD 미지원 
46               스냅드래곤778G/ 6nm/ microSD:최대1TB 
48                스냅드래곤695/ 6nm/ microSD:최대1TB 
49                스냅드래곤865+/ 7nm+/ microSD 미지원 
51           엑시노스7 7884/ 14nm/ microSD:최대512GB 
53     Helio G80(MT6769V)/ 12nm/ microSD:최대1TB 
57                A14 Bionic/ 5nm/ microSD 미지원 
58                  엑시노스2100/ 5nm/ micro

In [6]:
pd.set_option('display.max_rows', None)